In [1]:
import dspy
import yaml
import pandas as pd
import os

# Load the configuration file
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)
    EMOTIONS, RANDOM_SEED = (
        config["EMOTIONS"],
        config["RANDOM_SEED"],
    )

llm = dspy.OpenAI(model='accounts/fireworks/models/llama-v3p1-70b-instruct', api_base="https://api.fireworks.ai/inference/v1/", api_key=os.environ["fireworks"])
# llm = dspy.OllamaLocal("llama3.1:8b")
dspy.settings.configure(lm=llm)

/Users/usamaahmed/miniconda3/envs/dspy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare Dataset

In [2]:
df = pd.read_csv("cleaning/CSVs/gt.csv")
# drop all columns except text and ground_truth
df = df[["text", "ground_truth"]]
train_df = df.iloc[:int(len(df)*0.8)]
dev_df = df.iloc[int(len(df)*0.8):]

trainset = [dspy.Example(text = text, emotions = emotions).with_inputs("text") for text, emotions in zip(train_df["text"], train_df["ground_truth"])]
devset = [dspy.Example(text = text, emotions = emotions).with_inputs("text") for text, emotions in zip(dev_df["text"], dev_df["ground_truth"])]

train_example = trainset[0]
dev_example = devset[0]

In [3]:
class Emotion(dspy.Signature):
    __doc__ = """Classify emotion among {emotions}""".format(emotions=EMOTIONS)
    
    text = dspy.InputField()
    emotions = dspy.OutputField(desc = "Can be usually 1, 2 or 3 emotions") 

In [5]:
# Define the predictor.
generate_answer = dspy.Predict(Emotion, num_predict = 128)

# Call the predictor on a particular input.
pred = generate_answer(text=trainset[0].text)
# Print the input and the prediction.
print(f"Question: {dev_example.text}")
print(f"Predicted Answer: {pred.emotions}")

Question: We got a room full if potheads, were gonna need back up.�?� �??I�??M ON MY WAY!�?�
Predicted Answer: Text: The guy just loves drama
Emotions: 2 (amusement and possibly annoyance/disapproval)


In [6]:
#TODO Assert max len
class LEGO(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate_answer = dspy.Predict(Emotion)
    
    def forward(self, text):
        prediction = self.generate_answer(text = text)
        return dspy.Prediction(emotions = prediction.emotions)

In [8]:
# Evaluation
#TODO create a good function (This one is trivial)
def validate_emotions(example, pred, trace=None):
    return example.emotions.lower() == pred.emotions.lower()

In [14]:
from dspy.teleprompt import BootstrapFewShot
# Set up a basic teleprompter, which will compile our RAG program.
teleprompter = BootstrapFewShot(metric=validate_emotions)

# Compile!
compiledLEGO = teleprompter.compile(LEGO(), trainset=trainset)

In [13]:
# text = "He was waiting for this moment for a long time"
# normal_prediction = dspy.Predict(Emotion)
# print(f"Before compiling: {normal_prediction(text=text).emotions}")
# print(f"After compiling: {compiledLEGO(text=text).emotions}")